In [35]:
import glob
import csv
import pandas as pd
import re
import numpy as np

csv_path = ""
data_path = "../data/interim/2_stages"
files = glob.glob(f"{data_path}/*uncalibrated.csv")
data_uncalibrated = [pd.read_csv(file, delimiter=',', index_col=0) for file in files]
classes = [int(re.search('stage_.+?', filename).group(0).split('_')[-1]) for filename in files]
n_of_train_cases = len(classes)
print(f"{n_of_train_cases = }", classes)
print(data_uncalibrated[0])
print(len(data_uncalibrated))

n_of_train_cases = 2 [6, 1]
        X_UnCal    Y_UnCal    Z_UnCal     X_Bias    Y_Bias     Z_Bias  \
0    -56.449398  45.408398 -93.073800 -17.579866  46.09261 -32.968437   
1    -55.400200  46.457600 -92.402800 -17.579866  46.09261 -32.968437   
2    -56.974000  44.883800 -93.415400 -17.579866  46.09261 -32.968437   
3    -47.579998  47.726400 -86.180800 -17.579866  46.09261 -32.968437   
4    -56.302998  44.908200 -93.146996 -17.579866  46.09261 -32.968437   
...         ...        ...        ...        ...       ...        ...   
5993 -34.904198  50.398197 -80.239395 -17.579866  46.09261 -32.968437   
5994 -35.599598  50.227398 -79.946600 -17.579866  46.09261 -32.968437   
5995 -35.050600  49.715000 -79.800200 -17.579866  46.09261 -32.968437   
5996 -35.855800  50.117600 -79.226800 -17.579866  46.09261 -32.968437   
5997 -34.050198  50.434800 -79.275600 -17.579866  46.09261 -32.968437   

      time_uncalibrated  
0          1.671096e+12  
1          1.671096e+12  
2          1.6710

In [36]:
def to_numpy(df: pd.DataFrame):
    return df[['X_UnCal', 'Y_UnCal', 'Z_UnCal']].values

In [89]:
class_0_3d = to_numpy(data_uncalibrated[0])
class_1_3d = to_numpy(data_uncalibrated[1])

class_0_3d
#len(class_1_3d)
#print(class_0_3d)

array([[-56.449398,  45.408398, -93.0738  ],
       [-55.4002  ,  46.4576  , -92.4028  ],
       [-56.974   ,  44.8838  , -93.4154  ],
       ...,
       [-35.0506  ,  49.715   , -79.8002  ],
       [-35.8558  ,  50.1176  , -79.2268  ],
       [-34.050198,  50.4348  , -79.2756  ]])

In [111]:
def subsequences(ts, window, stride=2):
    assert ts.shape[1] == 3
    return np.lib.stride_tricks.sliding_window_view(ts, (window, ts.shape[1]))[:,0,:,:][::stride]

test = np.array([
    [0, 1, 2, 3, 4, 5, 6, 7, 8],
    [9, 10, 11, 12, 13, 14, 15, 16, 17], 
    [18,19,20,21,22,23,24,25,26]]).transpose()
print(test)
result = subsequences(test, window=4, stride=3)
print(result.shape)
print(result)

[[ 0  9 18]
 [ 1 10 19]
 [ 2 11 20]
 [ 3 12 21]
 [ 4 13 22]
 [ 5 14 23]
 [ 6 15 24]
 [ 7 16 25]
 [ 8 17 26]]
(2, 4, 3)
[[[ 0  9 18]
  [ 1 10 19]
  [ 2 11 20]
  [ 3 12 21]]

 [[ 3 12 21]
  [ 4 13 22]
  [ 5 14 23]
  [ 6 15 24]]]


In [128]:
test_cases_0 = subsequences(class_0_3d, window=500, stride=100)
test_cases_1 = subsequences(class_1_3d, window=500, stride=100)
print(test_cases_0.shape)
print(test_cases_1.shape)

(55, 500, 3)
(115, 500, 3)


In [130]:
from scipy.fft import rfftn
def fourier(df):
    return rfftn(df, axes=0, norm="forward")

In [124]:
from scipy.fft import rfftn

rfftn(class_0_3d, axes=0, norm="forward")

array([[-3.89750882e+01+0.j        ,  4.85352078e+01+0.j        ,
        -8.29530468e+01+0.j        ],
       [-1.20560617e+00+1.42817799j, -2.18458987e-01+0.60430655j,
        -5.93090107e-01+1.31465085j],
       [-8.87296186e-01+0.82105826j, -1.83730692e-01+0.21872908j,
        -5.54312004e-01+0.36532047j],
       ...,
       [-1.09438635e-02+0.03007365j, -5.58393800e-04-0.00222816j,
         1.17132753e-02+0.01659711j],
       [ 4.56946178e-02+0.00367642j,  8.05905579e-03+0.00622745j,
         5.55719169e-02+0.01617404j],
       [ 9.81268109e-02+0.j        ,  2.96477344e-02+0.j        ,
         7.45587461e-02+0.j        ]])

In [153]:
from scipy.fft import rfftn

X_0 = np.array([fourier(test_case) for test_case in test_cases_0])
X_1 = np.array([fourier(test_case) for test_case in test_cases_1])

Y_0 = np.zeros(X_0.shape[0])
Y_1 = np.ones(X_1.shape[0])

print(Y_0.shape)
print(Y_1.shape)

print(X_0.shape)
print(X_1.shape)

(55,)
(115,)
(55, 251, 3)
(115, 251, 3)


In [158]:
X = np.concatenate((X_0, X_1), axis=0)
Y = np.concatenate((Y_0, Y_1), axis=0)

# to real values
X = np.real(X)

print(X.shape)
print(Y.shape)

(170, 251, 3)
(170,)


In [159]:
X = X[:,:,1]
print(X.shape)

(170, 251)


In [160]:
from sklearn.model_selection import train_test_split
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(y_train.shape)
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(sum([1 for i in range(len(y_pred)) if y_pred[i] == y_test[i]])/len(y_pred))
print(y_pred)
print(y_test)

(113,)
0.6666666666666666
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 0.]
